In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=4))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [3]:
preprocessor = mz.models.DSSM.get_default_preprocessor(ngram_size=3)
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
valid_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 10688.44it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:02<00:00, 6461.84it/s]
Building Vocabulary from a datapack.: 100%|██████████| 2082963/2082963 [00:00<00:00, 4107566.35it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 11845.98it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:02<00:00, 6507.02it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 813967.84it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 9048.56it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:0

In [4]:
preprocessor.context

{'ngram_process_unit': <matchzoo.preprocessors.units.ngram_letter.NgramLetter at 0x13b274e10>,
 'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x10b1f3550>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x10b71ea20>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058,
 'ngram_vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x13e2a7f98>,
 'ngram_vocab_size': 9654}

In [5]:
triletter_callback = mz.dataloader.callbacks.Ngram(
    preprocessor, mode='aggregate')

trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=1,
    num_neg=4,
    callbacks=[triletter_callback]
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    callbacks=[triletter_callback]
)

In [6]:
padding_callback = mz.models.DSSM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=32,
    stage='train',
    resample=True,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=32,
    stage='dev',
    callback=padding_callback
)

In [7]:
model = mz.models.DSSM()

model.params['task'] = ranking_task
model.params['vocab_size'] = preprocessor.context['ngram_vocab_size']
model.params['mlp_num_layers'] = 3
model.params['mlp_num_units'] = 300
model.params['mlp_num_fan_out'] = 128
model.params['mlp_activation_func'] = 'relu'

model.build()

print(model, sum(p.numel() for p in model.parameters() if p.requires_grad))

DSSM(
  (mlp_left): Sequential(
    (0): Sequential(
      (0): Linear(in_features=9654, out_features=300, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (3): Sequential(
      (0): Linear(in_features=300, out_features=128, bias=True)
      (1): ReLU()
    )
  )
  (mlp_right): Sequential(
    (0): Sequential(
      (0): Linear(in_features=9654, out_features=300, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (3): Sequential(
      (0): Linear(in_features=300, out_features=128, bias=True)
      (1): ReLU()
    )
  )
  (out): Linear(in_features=1, out_features=1, bias=True)
)

In [8]:
optimizer = torch.optim.Adam(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [9]:
trainer.run()

[Iter-32 Loss-1.521]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5219 - normalized_discounted_cumulative_gain@5(0.0): 0.5864 - mean_average_precision(0.0): 0.5432



[Iter-64 Loss-1.343]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4888 - normalized_discounted_cumulative_gain@5(0.0): 0.5738 - mean_average_precision(0.0): 0.5166



[Iter-96 Loss-1.320]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4447 - normalized_discounted_cumulative_gain@5(0.0): 0.5118 - mean_average_precision(0.0): 0.4652



[Iter-128 Loss-1.262]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4977 - normalized_discounted_cumulative_gain@5(0.0): 0.5674 - mean_average_precision(0.0): 0.5216



[Iter-160 Loss-1.213]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4819 - normalized_discounted_cumulative_gain@5(0.0): 0.5456 - mean_average_precision(0.0): 0.4997



[Iter-192 Loss-1.186]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4921 - normalized_discounted_cumulative_gain@5(0.0): 0.5655 - mean_average_precision(0.0): 0.5069



[Iter-224 Loss-1.177]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4859 - normalized_discounted_cumulative_gain@5(0.0): 0.5583 - mean_average_precision(0.0): 0.5056



[Iter-256 Loss-1.137]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4789 - normalized_discounted_cumulative_gain@5(0.0): 0.5499 - mean_average_precision(0.0): 0.4926



[Iter-288 Loss-1.126]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4967 - normalized_discounted_cumulative_gain@5(0.0): 0.5725 - mean_average_precision(0.0): 0.5274



[Iter-320 Loss-1.092]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4868 - normalized_discounted_cumulative_gain@5(0.0): 0.55 - mean_average_precision(0.0): 0.506

Cost time: 1625.7705359458923s
